# Multi Task Model

## Imports

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from itertools import product

# Tensorflow imports
import tensorflow as tf
from tensorflow import keras
import keras.backend as K
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications import ResNet50

from loadData_alt import createDataset

## Model parameters

In [2]:
# Log parameters
model_name = 'mtm_classification_cascade'
savedModelPath = f'../../log/saved_models/{model_name}'
tb_log_dir = f'../../log/tensorboard/{model_name}'
cp_filepath = f'../../log/cps/{model_name}/'

if not os.path.exists(cp_filepath):
    os.makedirs(cp_filepath)

cp_filepath += 'latest_weights.h5'

# Dynamic hyperparameters
learningRate = 0.001
doDataAugmentation = True
dropoutRate = 0.25
width_multiplier = 1
depth_multiplier = 1

# Training parameters
batch_size = 32
epochs = 10

## Model callbacks

In [3]:
callbacks = [
    # Checkpoint callback                    
    keras.callbacks.ModelCheckpoint(
                    filepath=cp_filepath, 
                    verbose=1, 
                    save_weights_only=True),

    # Tensorboard callback
    keras.callbacks.TensorBoard(log_dir=tb_log_dir, histogram_freq=1),

    # Early Stopping callback
    # keras.callbacks.EarlyStopping(
    #                 monitor="val_loss",
    #                 patience=2,
    #                 verbose=1)
]

## Data parameters

In [4]:
image_height = 160
image_width = 160

use_age_groups = False
num_age_classes = 8 if use_age_groups else 122

age_classes_dict = {
    0 : "0-13",
    1 : "14-19",
    2 : "20-29",
    3 : "30-39",
    4 : "40-49",
    5 : "50-59",
    6 : "60+",
    7 : "-"
}

## Data Augmentation

In [5]:
data_augmentation = keras.Sequential(
    [
        keras.layers.RandomRotation(0.1),
        keras.layers.RandomBrightness(0.2),
    ]
)

## Creating the training datasets

In [6]:
training_ds, validation_ds = createDataset('../../data/m3/training', (image_height, image_width), batch_size, 0.2, use_age_groups=use_age_groups)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


## Load the ResNet model

In [7]:
# Loading either the MobileNet architecture model or the previously saved model, and freeze it for transfer learning
mobilenet = ResNet50(
                input_shape=(image_height, image_width, 3), # Optional shape tuple, only to be specified if include_top is False
                # alpha=width_multiplier, # Controls the width of the network. (Width multiplier)
                # depth_multiplier=depth_multiplier, # Depth multiplier for depthwise convolution. (Resolution multiplier)
                # dropout=dropoutRate, # Dropout rate. Default to 0.001.
                weights="imagenet",
                # input_tensor=None,
                # pooling='avg', # Optional pooling mode for feature extraction when include_top is False. (None, avg, max)
                include_top=False
                )
           
# Freeze the base model
mobilenet.trainable = False

inputs = keras.Input(shape=(image_height, image_width, 3))

# Data Augmentation on input
if(doDataAugmentation):
    inputs = data_augmentation(inputs)

# Running base model in inference mode
base_model = mobilenet(inputs, training=False)
base_model = keras.layers.GlobalAveragePooling2D()(base_model)
base_model = keras.layers.Dense(1024)(base_model)
base_model = keras.layers.Dropout(0.3)(base_model)

## Creating Task 1 (Face Detection) Top Model

In [8]:
# Add Dense layer
face_head = tf.keras.layers.Dense(256, activation='relu')(base_model)
face_head = keras.layers.BatchNormalization()(face_head)
face_head = keras.layers.Activation('relu')(face_head)
face_head = keras.layers.Dropout(0.2)(face_head)
face_head = tf.keras.layers.Dense(128, activation='relu')(face_head)

# Final layer for binary classification
face_outputs = keras.layers.Dense(1, activation='sigmoid')(face_head)

In [9]:
multiply_layer = keras.layers.Multiply()([base_model, face_outputs])
multiply_layer = keras.layers.Dense(512, activation='relu')(multiply_layer)

## Creating Task 2 (Mask Detection) Top Model

In [10]:
# Add Dense layer
mask_head = tf.keras.layers.Dense(128, activation='relu')(multiply_layer)
mask_head = keras.layers.BatchNormalization()(mask_head)
mask_head = keras.layers.Activation('relu')(mask_head)
mask_head = keras.layers.Dropout(0.2)(mask_head)
mask_head = tf.keras.layers.Dense(64, activation='relu')(mask_head)

# Final layer for binary classification
mask_outputs = keras.layers.Dense(1, activation='sigmoid', name='mask_output')(mask_head)

## Creating Task 3 (Age Prediction) Top Model

In [11]:
# Add Dense layer
age_head = tf.keras.layers.Dense(256, activation='relu')(multiply_layer)
age_head = keras.layers.BatchNormalization()(age_head)
age_head = keras.layers.Activation('relu')(age_head)
age_head = keras.layers.Dropout(0.2)(age_head)
age_head = tf.keras.layers.Dense(128, activation='relu')(age_head)
age_head = tf.keras.layers.Dense(64, activation='relu')(age_head)

# Final layer for binary classification
age_outputs = keras.layers.Dense(num_age_classes, activation='softmax', name='age_output')(age_head)

In [12]:
face_outputs = keras.layers.Dense(1, activation='sigmoid', name='face_output')(multiply_layer)

## Creating and compiling the final model

In [13]:

model = keras.Model(inputs, [face_outputs, mask_outputs, age_outputs])
keras.utils.plot_model(model, to_file=f'{model_name}.png', show_layer_activations=True)

# Using a joint loss function for the three tasks:
# [ Loss = gamma * Loss_task1 + gamma * Loss_task2 + gamma * Loss_task3 ]
# Because every task is dependant on every other task, the model receives the loss of every task when gamma > 0

gamma = 1

model.compile(
      optimizer=keras.optimizers.Adam(), # Learning Rate?
            loss={
                  'face_output': 'binary_crossentropy', 
                  'mask_output': 'binary_crossentropy',
                  'age_output': 'categorical_crossentropy'
                  },
            loss_weights={
                  'face_output': 0.33 * gamma,
                  'mask_output': 0.33 * gamma,
                  'age_output': 0.33 * gamma
                  }, 
            metrics={
                  'face_output': 'binary_accuracy', 
                  'mask_output': 'binary_accuracy',
                  'age_output': 'categorical_accuracy'
                  },
)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 160, 160, 3  0           []                               
                                )]                                                                
                                                                                                  
 resnet50 (Functional)          (None, 5, 5, 2048)   23587712    ['input_3[0][0]']                
                                                                                                  
 global_average_pooling2d (Glob  (None, 2048)        0           ['resnet50[1][0]']               
 alAveragePooling2D)                                                                              
                                                                                              

## Training the model with the dataset

In [14]:
history = model.fit(
            training_ds,
            validation_data=validation_ds,
            batch_size=batch_size,
            epochs=epochs, 
            callbacks=callbacks,
        )

Epoch 1/10
353/353 [==============================] - ETA: 0s - loss: 0.7781 - face_output_loss: 0.0904 - mask_output_loss: 0.0655 - age_output_loss: 2.2019 - face_output_binary_accuracy: 0.9880 - mask_output_binary_accuracy: 0.9742 - age_output_categorical_accuracy: 0.4915
Epoch 1: saving model to ../../log/cps/mtm_classification_cascade\latest_weights.h5
353/353 [==============================] - 953s 3s/step - loss: 0.7781 - face_output_loss: 0.0904 - mask_output_loss: 0.0655 - age_output_loss: 2.2019 - face_output_binary_accuracy: 0.9880 - mask_output_binary_accuracy: 0.9742 - age_output_categorical_accuracy: 0.4915 - val_loss: 0.7732 - val_face_output_loss: 0.0509 - val_mask_output_loss: 0.0324 - val_age_output_loss: 2.2597 - val_face_output_binary_accuracy: 0.9887 - val_mask_output_binary_accuracy: 0.9918 - val_age_output_categorical_accuracy: 0.5089
Epoch 2/10
353/353 [==============================] - ETA: 0s - loss: 0.6658 - face_output_loss: 0.0216 - mask_output_loss: 0.0115 

## Save the model

In [15]:
model.save(savedModelPath)

INFO:tensorflow:Assets written to: ../../log/saved_models/mtm_classification_cascade\assets


INFO:tensorflow:Assets written to: ../../log/saved_models/mtm_classification_cascade\assets
